# Tarea 1: DS Lab
- Ricardo Arancibia
- José Díaz
- Javier Santibáñez

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## P1

In [107]:
df_all = pd.read_csv('data/raw/w{}/metrocuadrado_all_w{}.csv'.format(str(13),str(13)))
for NN in range(14,18):
    direc = 'data/raw/w{}/metrocuadrado_all_w{}.csv'.format(str(NN),str(NN))
    df_aux = pd.read_csv(direc)
    # df_all.merge(df_aux)
    df_all = pd.concat([df_all,df_aux], ignore_index= True)
df_all['Archivo'] = 'all' 
df_all.drop_duplicates(inplace=True, ignore_index= True)
print(df_all.shape)
df_all.head()

(16295, 9)


,property_type|rent_type|location,price,n_rooms,n_bath,surface,details,url,metrocuadrado_index,Archivo
0,"Casa en Arriendo, LA SOLEDAD NORTE Bogotá D.C..",$1.050.000,3,2,63.0 m2,Excelente casa cerca de la calle 80 y Avenida ...,https://www.metrocuadrado.com/inmueble/arriend...,25.6,all
1,"Casa en Arriendo, CIUDAD SALITRE SUR-ORIENTAL ...",$3.930.000,3,2,100.0 m2,Se arrienda casa bonita totalmente amoblada co...,https://www.metrocuadrado.com/inmueble/arriend...,57.3,all
2,"Casa en Arriendo, VILLA DE LOS ALPES II SECTOR...",$750.000,3,1,90.0 m2,CASA DE TRES NIVELES TRES ALCOBAS SALA COMEDOR...,https://www.metrocuadrado.com/inmueble/arriend...,12.2,all
3,"Casa en Arriendo, PARIS GAITAN Bogotá D.C..",$3.400.000,5,4,71.0 m2,casa ezquinera excelente ubicacion cinco alcob...,https://www.metrocuadrado.com/inmueble/arriend...,56.0,all
4,"Casa en Arriendo, BALCONES DE ORIENTE Bogotá D...",$1.560.000,4,3,96.0 m2,"Excelente casa de tres niveles, buena ubicació...",https://www.metrocuadrado.com/inmueble/arriend...,47.1,all


In [108]:
df_fur = pd.read_csv('data/raw/w{}/metrocuadrado_furnished_w{}.csv'.format(str(13),str(13)))
for NN in range(14,18):
    direc = 'data/raw/w{}/metrocuadrado_furnished_w{}.csv'.format(str(NN),str(NN))
    df_aux = pd.read_csv(direc)
    # df_fur.merge(df_aux)
    df_fur = pd.concat([df_fur,df_aux], ignore_index= True)
df_fur['Archivo'] = 'furnished'
df_fur.drop_duplicates(inplace=True, ignore_index= True)
print(df_fur.shape)
df_fur.head()

(2099, 9)


,property_type|rent_type|location,price,n_rooms,n_bath,surface,details,url,metrocuadrado_index,Archivo
0,"Casa en Arriendo, CIUDAD SALITRE SUR-ORIENTAL ...",$3.930.000,3.0,2.0,100.0 m2,Se arrienda casa bonita totalmente amoblada co...,https://www.metrocuadrado.com/inmueble/arriend...,57.3,furnished
1,"Casa en Venta Y Arriendo, BELLAVISTA Bogotá D.C..",$1.300.000,3.0,2.0,72.0 m2,"hermosa casa en venta y en arriendo en, mandal...",https://www.metrocuadrado.com/inmueble/venta-c...,41.4,furnished
2,"Casa en Arriendo, PORTALES DEL NORTE Bogotá D.C..",$1.800.000,3.0,3.0,76.81 m2,Estrato 4. CASA TRIPLEX!! Exterior . Cuenta co...,https://www.metrocuadrado.com/inmueble/arriend...,41.3,furnished
3,"Casa en Venta Y Arriendo, SANTA SOFIA Bogotá D...",$1.500.000,5.0,3.0,76.22 m2,"Casa en Santa Sofia, excelente ubicación sobre...",https://www.metrocuadrado.com/inmueble/venta-c...,34.2,furnished
4,"Casa en Venta Y Arriendo, URBANIZACION SABANA ...",$145.000.000,3.0,2.0,77.0 m2,"Casa esquinera, de tres niveles, se venden mue...",https://www.metrocuadrado.com/inmueble/venta-c...,62.5,furnished


In [154]:
df = pd.concat([df_all,df_fur],ignore_index= True)
print(df.shape)
df.head()

(18394, 9)


,property_type|rent_type|location,price,n_rooms,n_bath,surface,details,url,metrocuadrado_index,Archivo
0,"Casa en Arriendo, LA SOLEDAD NORTE Bogotá D.C..",$1.050.000,3,2,63.0 m2,Excelente casa cerca de la calle 80 y Avenida ...,https://www.metrocuadrado.com/inmueble/arriend...,25.6,all
1,"Casa en Arriendo, CIUDAD SALITRE SUR-ORIENTAL ...",$3.930.000,3,2,100.0 m2,Se arrienda casa bonita totalmente amoblada co...,https://www.metrocuadrado.com/inmueble/arriend...,57.3,all
2,"Casa en Arriendo, VILLA DE LOS ALPES II SECTOR...",$750.000,3,1,90.0 m2,CASA DE TRES NIVELES TRES ALCOBAS SALA COMEDOR...,https://www.metrocuadrado.com/inmueble/arriend...,12.2,all
3,"Casa en Arriendo, PARIS GAITAN Bogotá D.C..",$3.400.000,5,4,71.0 m2,casa ezquinera excelente ubicacion cinco alcob...,https://www.metrocuadrado.com/inmueble/arriend...,56.0,all
4,"Casa en Arriendo, BALCONES DE ORIENTE Bogotá D...",$1.560.000,4,3,96.0 m2,"Excelente casa de tres niveles, buena ubicació...",https://www.metrocuadrado.com/inmueble/arriend...,47.1,all


In [155]:
df.dropna(subset=['price','n_rooms','n_bath','surface'], inplace=True)
df.reset_index(drop=True,inplace=True)
print(df.shape)

(18143, 9)


In [156]:
df['price'] = [x.replace('$','') for x in df['price']]
df['price'] = [x.replace('.','') for x in df['price']]
df['surface'] = [x.replace(' m2','') for x in df['surface']]
df['price'] = df['price'].astype('float64')
df['surface'] = df['surface'].astype('float64')
# df.head()

In [157]:
df_aux = df['property_type|rent_type|location'].str.split(pat=',',expand =True)
df_aux[0] = [x.replace(' ',', ', 1) for x in df_aux[0]]
df_aux = pd.concat([df_aux[0].str.split(pat=',',expand =True), df_aux[1]], axis=1, ignore_index=True)
df_aux.columns = ['property type','rent_type','location']
df_aux['rent_type'] = [x.replace('en ','') for x in df_aux['rent_type']]
# df_aux
df = pd.concat([df_aux,df.drop(columns=['property_type|rent_type|location'])], axis=1)
df.head()

,property type,rent_type,location,price,n_rooms,n_bath,surface,details,url,metrocuadrado_index,Archivo
0,Casa,Arriendo,LA SOLEDAD NORTE Bogotá D.C..,1050000.0,3,2,63.0,Excelente casa cerca de la calle 80 y Avenida ...,https://www.metrocuadrado.com/inmueble/arriend...,25.6,all
1,Casa,Arriendo,CIUDAD SALITRE SUR-ORIENTAL Bogotá D.C..,3930000.0,3,2,100.0,Se arrienda casa bonita totalmente amoblada co...,https://www.metrocuadrado.com/inmueble/arriend...,57.3,all
2,Casa,Arriendo,VILLA DE LOS ALPES II SECTOR Bogotá D.C..,750000.0,3,1,90.0,CASA DE TRES NIVELES TRES ALCOBAS SALA COMEDOR...,https://www.metrocuadrado.com/inmueble/arriend...,12.2,all
3,Casa,Arriendo,PARIS GAITAN Bogotá D.C..,3400000.0,5,4,71.0,casa ezquinera excelente ubicacion cinco alcob...,https://www.metrocuadrado.com/inmueble/arriend...,56.0,all
4,Casa,Arriendo,BALCONES DE ORIENTE Bogotá D.C..,1560000.0,4,3,96.0,"Excelente casa de tres niveles, buena ubicació...",https://www.metrocuadrado.com/inmueble/arriend...,47.1,all
